# Diabetes Risk Prediction with Decision Tree from Scratch

Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import KFold

In [2]:
dataset = pd.read_csv('diabetes_data_upload.csv')

### Discretization process on continuous attribute

Friend of mine which is med student recommend me to split data this way. It gives %100 accuracy without shuffleing and Kfolds

Ages splitted to 5 and labeled as 1,2,3,4,5

In [3]:
max_age = dataset['Age'].max() #max age in dataset
min_age = dataset['Age'].min() #min age in dataset
dataset['Age']=pd.cut(x = dataset['Age'],bins = [min_age,30,35,40,55,max_age],labels = [1, 2, 3, 4, 5])
#dataset=dataset.sample(frac=1) #shuffling data

Calculating the entropy

In [4]:
def entropy(target_col):

    elements,counts = np.unique(target_col,return_counts = True) #finding unique elements

    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))]) #calculating entropy

    return entropy

### Calculation of information gain
Finding the imformation gain for every feature. Imformation Gain(Feature) is simply Entropy(Whole dataset) - Entropy(Feature)

In [5]:
def info_gain(data,split_attribute_name,target_name="class"):

    total_entropy = entropy(data[target_name]) #total entropy of dataset
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)

    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])

    information_gain = total_entropy - Weighted_Entropy #calculating information gain
    return information_gain

### ID3 Algo
This algorithm will build tree recursively. We have 3 base cases for stopping recursion

Base case 1: If all rows in target feature same value

Base case 2: When dataset ends

Base case 3: When dataset no longer split

##### Building the tree
Create base node

Find whole entropy then calculate info gain for each attribute and pick the largest gain

Assign the node of the feature. Grow for each feature value an outgoing branch and add unlabeled nodes at the and

Remove the feature with highest info gain

Repeat until base case

In [6]:
def id3(data,original_data,features,target_label="class",parent_node_class = None):

    if len(np.unique(data[target_label])) <= 1: #base cases
        return np.unique(data[target_label])[0]

    elif len(data)==0:
        return np.unique(original_data[target_label])[np.argmax(np.unique(original_data[target_label],return_counts=True)[1])]

    elif len(features) ==0:
        return parent_node_class



    else: #else build the tree

        parent_node_class = np.unique(data[target_label])[np.argmax(np.unique(data[target_label],return_counts=True)[1])]

        item_values = [info_gain(data,feature,target_label) for feature in features] #return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        tree = {best_feature:{}}


        features = [i for i in features if i != best_feature] #remove best inforamtion gain

        for value in np.unique(data[best_feature]):
            value = value

            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = id3(sub_data,dataset,features,target_label,parent_node_class) #recursion

            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree

        return(tree)

Function that returns either prediction is correct or not

There is a problem here. Unseen query it may happen that the feature values of these query are not existing in our tree model because non of the training instances has had such a value for this specific feature. If this happens we decided to assign value to Negative. These are our examples of missclassified samples. We can simply drop this samples to improve our accuracy.

In [7]:
def predict(query,tree,default = "Negative"):

    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

Function that stores predictions. Scikit functions uses this returned values to calculate accuracy, precision etc.

In [8]:
def tester(data,tree):

    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"])

    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,"Negative")
        #print(predicted.iloc[0:1,i])
    return predicted["predicted"]

Splitting data into 5 folds with scikit

In [9]:
X = dataset.iloc[:,:]
y = dataset.iloc[:,-1]
k = 5
kf = KFold(n_splits=k, random_state=None)

There is no general rule for confusion matrix. This creates confusion. So I picked labels myself. Matrix structre showed below.

In [10]:
avg_accuracy = 0
avg_precision = 0
avg_recall = 0
avg_f1 = 0

for train_index , test_index in kf.split(X):
    X_train= X.iloc[train_index,:]
    y_train = X.iloc[test_index,:]

    tree = id3(X_train,X_train,X_train.columns[:-1])
    tree_calc = tester(y_train,tree)

    confusion_mtx = confusion_matrix(y_train["class"], tree_calc, labels=['Positive','Negative'])
    print("Confusion Matrix")
    print(confusion_mtx)
    """
    matrix structure
    TP | FN
    --   --
    FP | TN
    """

    avg_accuracy += accuracy_score(y_train["class"], tree_calc)
    avg_precision += precision_score(y_train["class"], tree_calc, pos_label='Positive')
    avg_recall += recall_score(y_train["class"], tree_calc, pos_label='Positive')
    avg_f1 += f1_score(y_train["class"], tree_calc, pos_label='Positive')
    print("Accuracy:", accuracy_score(y_train["class"], tree_calc))
    print("Precision:", precision_score(y_train["class"], tree_calc, pos_label='Positive'))
    print("Recall:", recall_score(y_train["class"], tree_calc, pos_label='Positive'))
    print("F1 score:", f1_score(y_train["class"], tree_calc, pos_label='Positive'))
    print()

print("Average Accuracy", avg_accuracy/5)
print("Average Precision", avg_precision/5)
print("Average Recall", avg_recall/5)
print("Average F1 Score", avg_f1/5)


Confusion Matrix
[[95  9]
 [ 0  0]]
Accuracy: 0.9134615384615384
Precision: 1.0
Recall: 0.9134615384615384
F1 score: 0.9547738693467337

Confusion Matrix
[[88  8]
 [ 0  8]]
Accuracy: 0.9230769230769231
Precision: 1.0
Recall: 0.9166666666666666
F1 score: 0.9565217391304348

Confusion Matrix
[[33  0]
 [ 4 67]]
Accuracy: 0.9615384615384616
Precision: 0.8918918918918919
Recall: 1.0
F1 score: 0.9428571428571428

Confusion Matrix
[[31  0]
 [10 63]]
Accuracy: 0.9038461538461539
Precision: 0.7560975609756098
Recall: 1.0
F1 score: 0.8611111111111112

Confusion Matrix
[[56  0]
 [ 0 48]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0

Average Accuracy 0.9403846153846154
Average Precision 0.9295978905735003
Average Recall 0.966025641025641
Average F1 Score 0.9430527724890844


True Positives (TP) - These are the correctly predicted positive values which means that the value of actual class is positive and the value of predicted class is also positive

True Negatives (TN) - These are the correctly predicted negative values which means that the value of actual class is negative and value of predicted class is also negative

False Positives (FP) – When actual class is negative and predicted class is positive. E.g. if actual class says this passenger healty but predicted class tells you that this person has a diabet.

False Negatives (FN) – When actual class is positive but predicted class in negative. E.g. if actual class value indicates that this person has a diabet and predicted class tells you that person is healty.

#### Accuracy
Accuracy is the most intuitive performance measure and it is simply a ratio of correctly predicted observation to the total observations. Accuracy is a great measure but only when we have symmetric datasets where values of false positive and false negatives are almost same. In our first 4 folds we dont have a symetric FP and FN. So accuracy is not a good model for our data.

#### Precision
Precision is the ratio of correctly predicted positive observations to the total predicted positive observations. The question that this metric answer is of all passengers that labeled as positive, how many actually positive? High precision relates to the low false positive rate. In first two folds we have a 1.0 precision which is nice. The precision score is a useful measure of the success of prediction when the classes are very imbalanced.

#### Recall
Recall score represents the model’s ability to correctly predict the positives out of actual positives. This is unlike precision which measures how many predictions made by models are actually positive out of all positive predictions made. It measures how good our machine learning model is at identifying all actual positives out of all positives that exist within a dataset. In fold 3 and 4 we have a 1.0 recall score. Recall score is a useful measure of success of prediction when the classes are very imbalanced.

#### F1
F1 Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. Intuitively it is not as easy to understand as accuracy, F1 is usually more useful than accuracy when especially we have an uneven class distribution. F1 score is more usefull than accuracy for our data. It balances precision and recall. 

5th folds predicts %100 true. We can analyze train and set set of this fold and can understand better for improving general prediction percentage.

# Pruning Decision Tree

Pruning is a technique that is used to reduce overfitting. Pruning also simplifies a decision tree by removing the weakest rules. Pruning is often distinguished into:
Pre-pruning (early stopping) stops the tree before it has completed classifying the training set,
Post-pruning allows the tree to classify the training set perfectly and then prunes the tree.

Travel all twigs in the tree

Find the twig with the least Information Gain

Remove all child nodes of the twig

Relabel twig as a Positive or Negative leaf

Measure the accuracy value of your decision tree model with removed
twig on the validation set (”Current Accuracy”)

If ”Current Accuracy ≥ Last Accuracy” : Jump to ”Step1”
Else : Revert the last changes done in Step 3,4 and then terminate


Pruning Function

In [11]:
def post_pruning(tree, df_train, df_val):

    question = list(tree.keys())[0]
    yes_answer, no_answer = tree[question]

    # base case
    if not isinstance(yes_answer, dict) and not isinstance(no_answer, dict):
        return pruning_result(tree, df_train, df_val)

    # recursive part
    else:
        df_train_yes, df_train_no = filter_df(df_train, question)
        df_val_yes, df_val_no = filter_df(df_val, question)

        if isinstance(yes_answer, dict):
            yes_answer = post_pruning(yes_answer, df_train_yes, df_val_yes)

        if isinstance(no_answer, dict):
            no_answer = post_pruning(no_answer, df_train_no, df_val_no)

        tree = {question: [yes_answer, no_answer]}

        return pruning_result(tree, df_train, df_val)

Helper Functions to pruning

In [12]:
def determine_leaf(df_train):

    return info_gain(df_train, df_train.columns[:-1], "class")

def determine_min_info_gain(df_val, tree):
    predictions = tester(df_val, tree)
    actual_values = df_val.label

    return predictions - actual_values


def pruning_result(tree, df_train, df_val):

    leaf = determine_leaf(df_train)
    errors_leaf = determine_min_info_gain(df_val, leaf)
    errors_decision_node = determine_min_info_gain(df_val, tree)

    if errors_leaf <= errors_decision_node:
        return leaf
    else:
        return tree

def filter_df(df, question):
    feature, comparison_operator, value = question.split()

    positive = df[df[feature].astype(str) == value]
    negative  = df[df[feature].astype(str) != value]

    return positive, negative